In [1]:
import random
import hashlib
import math

def hash_value(x, seed):
    random.seed(seed)
    salt = str(random.random())
    h = hashlib.sha1((str(x) + salt).encode()).hexdigest()
    return bin(int(h, 16))[2:]   # binary string

def trailing_zeros(binary_str):
    return len(binary_str) - len(binary_str.rstrip('0'))

def flajolet_martin(stream, num_hashes=10):
    max_trailing_list = []

    for i in range(num_hashes):
        max_trailing = 0
        for item in stream:
            h = hash_value(item, seed=i)
            tz = trailing_zeros(h)
            max_trailing = max(max_trailing, tz)
        max_trailing_list.append(max_trailing)

    # FM estimate
    avg_r = sum(max_trailing_list) / len(max_trailing_list)
    estimate = 2 ** avg_r
    return estimate, max_trailing_list


# ---- RUN TEST ----
stream = ["apple", "banana", "apple", "orange", "banana", "grapes", "apple"]
estimate, debug = flajolet_martin(stream)

print("Approx Distinct Count:", estimate)
print("Actual Distinct Count:", len(set(stream)))
print("Hash-based R Values:", debug)


Approx Distinct Count: 4.59479341998814
Actual Distinct Count: 4
Hash-based R Values: [3, 0, 4, 5, 2, 4, 1, 2, 0, 1]


In [2]:
from itertools import combinations
from collections import defaultdict

def get_support(transactions, itemset):
    count = sum(1 for transaction in transactions if itemset.issubset(transaction))
    return count

def apriori(transactions, min_support):
    transactions = list(map(set, transactions))
    items = sorted({item for t in transactions for item in t})

    # Step 1: L1
    L1 = []
    for item in items:
        if get_support(transactions, {item}) >= min_support:
            L1.append(frozenset([item]))

    L = [L1]
    k = 2
    
    while True:
        # Candidate generation
        candidates = []
        prev_L = L[-1]

        for a in prev_L:
            for b in prev_L:
                union = a | b
                if len(union) == k and union not in candidates:
                    candidates.append(union)

        # Prune
        Lk = []
        for c in candidates:
            if get_support(transactions, c) >= min_support:
                Lk.append(c)

        if not Lk:
            break
        
        L.append(Lk)
        k += 1
    
    return L


# ---- TEST EXAMPLE ----
transactions = [
    ["bread", "milk"],
    ["bread", "diaper", "beer", "egg"],
    ["milk", "diaper", "beer", "coke"],
    ["bread", "milk", "diaper", "beer"],
    ["bread", "milk", "diaper", "coke"],
]

min_support = 2

frequent_itemsets = apriori(transactions, min_support)
print("Frequent Itemsets:")
for level in frequent_itemsets:
    print(level)


Frequent Itemsets:
[frozenset({'beer'}), frozenset({'bread'}), frozenset({'coke'}), frozenset({'diaper'}), frozenset({'milk'})]
[frozenset({'beer', 'bread'}), frozenset({'beer', 'diaper'}), frozenset({'beer', 'milk'}), frozenset({'diaper', 'bread'}), frozenset({'milk', 'bread'}), frozenset({'coke', 'diaper'}), frozenset({'coke', 'milk'}), frozenset({'milk', 'diaper'})]
[frozenset({'beer', 'diaper', 'bread'}), frozenset({'beer', 'milk', 'diaper'}), frozenset({'milk', 'diaper', 'bread'}), frozenset({'coke', 'milk', 'diaper'})]


In [3]:
from collections import defaultdict, Counter

class FPNode:
    def __init__(self, item, count, parent):
        self.item = item
        self.count = count
        self.parent = parent
        self.children = {}

class FPTree:
    def __init__(self, transactions, min_support):
        self.min_support = min_support
        self.header = {}
        self.root = FPNode(None, 1, None)

        # Count item frequencies
        freq = Counter()
        for t in transactions:
            for item in t:
                freq[item] += 1
        
        # Filter by support
        self.freq_items = {item: cnt for item, cnt in freq.items() if cnt >= min_support}

        for t in transactions:
            sorted_items = sorted([x for x in t if x in self.freq_items],
                                  key=lambda x: self.freq_items[x],
                                  reverse=True)
            self.insert_tree(sorted_items, self.root)

    def insert_tree(self, items, node):
        if not items:
            return

        first = items[0]
        if first not in node.children:
            new_child = FPNode(first, 1, node)
            node.children[first] = new_child

            # Update header
            if first not in self.header:
                self.header[first] = []
            self.header[first].append(new_child)
        else:
            node.children[first].count += 1

        self.insert_tree(items[1:], node.children[first])


def fp_growth(tree, prefix, frequent_patterns):
    for item in tree.header:
        new_pattern = prefix + [item]
        frequent_patterns.append(new_pattern)
    
    return frequent_patterns


# TEST
transactions = [
    ["bread", "milk"],
    ["bread", "diaper", "beer"],
    ["milk", "diaper", "beer", "coke"],
    ["bread", "milk", "diaper", "beer"],
]

tree = FPTree(transactions, min_support=2)
patterns = fp_growth(tree, [], [])

print("Frequent Patterns:", patterns)


Frequent Patterns: [['bread'], ['milk'], ['diaper'], ['beer']]


In [5]:
import mmh3
from bitarray import bitarray

class BloomFilter:
    def __init__(self, size=100, hash_count=3):
        self.size = size
        self.hash_count = hash_count
        self.bit_array = bitarray(size)
        self.bit_array.setall(0)

    def add(self, item):
        for i in range(self.hash_count):
            digest = mmh3.hash(item, i) % self.size
            self.bit_array[digest] = 1

    def check(self, item):
        for i in range(self.hash_count):
            digest = mmh3.hash(item, i) % self.size
            if self.bit_array[digest] == 0:
                return False
        return True


# ---- TEST BLOOM FILTER ----
bf = BloomFilter(size=50, hash_count=3)
items = ["apple", "banana", "orange"]

for item in items:
    bf.add(item)

print("apple in set?", bf.check("apple"))
print("grapes in set?", bf.check("grapes"))  # maybe false/true due to false-positive


apple in set? True
grapes in set? False
